# Accessing Earth System Data from Cloud Storage
This guide will help you get started with accessing and analyzing large-scale earth system and weather datasets from various cloud providers using Python.

## Setup and Installation

First, you will need to add some packages I omitted from the codespace install the required packages:

```bash
pip install fsspec s3fs gcsfs adlfs h5netcdf
pip install planetary-computer stackstk pystac-client
```

## Basic Cloud Storage Access

### AWS S3


In [ ]:
import s3fs
import xarray as xr

fs = s3fs.S3FileSystem(anon=True)  # Use anon=True for public datasets

year = '2020'
doy = '018'
# Example: Access NOAA GOES-17 data
files= fs.glob(f's3://noaa-goes17/ABI-L2-SSTF/{year}/{doy}/*/*.nc')
with fs.open(files[0]) as f:
    ds = xr.open_dataset(f)
ds

## Google Cloud Storage (GCS)

In [ ]:
import gcsfs
import xarray as xr

# Initialize GCS filesystem
fs = gcsfs.GCSFileSystem(token='anon')  # Use token='anon' for public datasets

# Example: Access ERA5 data
path = 'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-1440x721.zarr'

ds = xr.open_dataset(path, engine='zarr')
ds = ds.chunk('auto')
ds

## From Azure via Planetary Computer

In [10]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("daymet-daily-hi")
collection

<CollectionClient id=daymet-daily-hi>

In [11]:
asset = collection.assets["zarr-abfs"]

In [12]:
list(asset.extra_fields.keys())

['xarray:open_kwargs', 'xarray:storage_options']

In [13]:
import xarray as xr

ds = xr.open_zarr(
    asset.href,
    **asset.extra_fields["xarray:open_kwargs"],
    storage_options=asset.extra_fields["xarray:storage_options"]
)
ds

<xarray.Dataset>
Dimensions:                  (time: 14965, y: 584, x: 284, nv: 2)
Coordinates:
    lat                      (y, x) float32 dask.array<chunksize=(584, 284), meta=np.ndarray>
    lon                      (y, x) float32 dask.array<chunksize=(584, 284), meta=np.ndarray>
  * time                     (time) datetime64[ns] 1980-01-01T12:00:00 ... 20...
  * x                        (x) float32 -5.802e+06 -5.801e+06 ... -5.519e+06
  * y                        (y) float32 -3.9e+04 -4e+04 ... -6.21e+05 -6.22e+05
Dimensions without coordinates: nv
Data variables:
    dayl                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    lambert_conformal_conic  int16 ...
    prcp                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    srad                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    swe                      (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    time_bnds                (time, nv) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    tmin                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    vp                       (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
    yearday                  (time) int16 dask.array<chunksize=(365,), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.6
    Version_data:      Daymet Data Version 4.0
    Version_software:  Daymet Software Version 4.0
    citation:          Please see http://daymet.ornl.gov/ for current Daymet ...
    references:        Please see http://daymet.ornl.gov/ for current informa...
    source:            Daymet Software Version 4.0
    start_year:        1980